In [57]:
import pandas as pd
from pathlib import Path
import sys
import os
import glob
import doctest

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

import importlib
import ParticleFrames
importlib.reload(ParticleFrames)

ParticleFrame = ParticleFrames.ParticleFrame
EnsembleFrames = ParticleFrames.EnsembleFrames
DesPartFrame = ParticleFrames.DesPartFrame

mid_sorted = EnsembleFrames.mid_sorted
tail_sorted = EnsembleFrames.tail_sorted

In [58]:
result = doctest.testmod(ParticleFrames)

In [61]:
tpdir = r'../data/enstest/'
tpfs = sorted(glob.glob(os.path.join(tpdir,'*ensemble*')),key=tail_sorted)
des = glob.glob(tpdir+'*destroy*')[0]
ens = EnsembleFrames(tpfs)
desf = DesPartFrame(des)

In [15]:
f1 = lambda df: df.get_indexed(desf.get_erange(1,100)['index'].compute())
ens.plot_yproj(interval=10, hdf=f1)

Reach the ParticleFrame with no data: ../data/enstest\enstest_ensemble_001020.csv


In [62]:
singlef = ens.derive_single_traj(1862)

No more data from the index: 1862 in the file: ../data/enstest\enstest_ensemble_000151.csv


In [63]:
singlef.plot_traj(bin=10)

dek1: 151.30583502934147
dek2: -0.5507048907098395
dek3: 0.19289253772093906


In [21]:
desf.plot_bottom()

### Test access and calculation speed based on pd.dataframe, dd.dataframe and hdf5

In [78]:
import pandas as pd
from dask import dataframe as dd
tfile = '../data/enstest_ensemble_000000.csv'
def test_pd_read_csv():
    dft = pd.read_csv(tfile)
    attr1 = dft[' x1']
    attr2 = dft[' x2']
def test_dd_read_csv():
    dft = dd.read_csv(tfile)
    dft.repartition(npartitions=10)
    attrs = dft[[' x1',' x2', ' usrpl01', ' time']].compute()
    attr1 = attrs[' x1']
    attr2 = attrs[' x2']

In [79]:
from line_profiler import LineProfiler
profiler = LineProfiler()
profiler.add_function(test_pd_read_csv)
profiler.add_function(test_dd_read_csv)
profiler.runcall(test_pd_read_csv)
profiler.runcall(test_dd_read_csv)
profiler.print_stats()

KeyError: "None of [Index(['x1', 'x2', 'x3'], dtype='object')] are in the [columns]"